In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [27]:
def compare_matches(img1, img2, center1, center2, patch_size=11):
    # TO DO: fix index out of img size
    height, width = img1.shape
    patch_half_size = patch_size // 2
    
    patch_1 = img1[center1[1]-patch_half_size:center1[1]+patch_half_size:]\
        [center1[0]-patch_half_size:center1[0]+patch_half_size:]
    patch_2 = img2[center2[1]-patch_half_size:center2[1]+patch_half_size:]\
        [center2[0]-patch_half_size:center2[0]+patch_half_size:]
    # assume everything is fine with patch
    
    return np.sum((patch_1-patch_2)**2)    
        
def patch_match(img1, img2, patch_size=11):
    height, width = img1.shape
    # create random ofset with the shape h*w*2 (dy,dx)
    match_x = np.random.randint(0, width, size=(height,width))
    match_y = np.random.randint(0, height, size=(height,width))
    
    iteration = 0
    
    while iteration<6:
        # choose direction of propagation
        if iteration % 2 == 0:
            iter_i = range(height)
            iter_j = range(width)
            edge_i = 0
            edge_j = 0
            delta = -1
        else:
            iter_i = range(height-1, -1, -1)
            iter_j = range(width-1, -1, -1)
            edge_i = height-1
            edge_j = width-1
            delta = 1
           
        # propagate
        for i in iter_i:
            for j in iter_j:
                current_x, current_y = match_x[i][j], match_y[i][j]
                candidates = [[current_x, current_y]]
                if i != edge_i:
                    candidates.append(match_x[i+delta][j], match_y[i+delta][j])
                if j != edge_j:
                    candidates.append(match_x[i][j+delta], match_y[i][j+delta])
                if len(candidates) == 1:
                    continue

                distances = [compare_matches(img1, img2, [i, j], candidates[k], patch_size) for k in range(len(candidates))]
                min_dist_index = np.argmin(distances)
                top_candidate = candidates[min_dist_index]

                match_x[i][j] = top_candidate[0]
                match_y[i][j] = top_candidate[1]        
        
                # random search around
                wa_x = width // 2
                wa_y = height // 2
                
                while wa_x >= 1 and wa_y >= 1:
                    deltas = [-1,1] 
                    candidate_x = current_x + wa_x * deltas[np.randint(0,2)]
                    candidate_y = current_y + wa_y * deltas[np.randint(0,2)]
                    
                    candidate_x = candidate_x % width
                    candidate_y = candidate_y % height
                    
                    distance = compare_matches(img1, img2, [i, j], [candidate_x, candidate_y], patch_size)
                    if distance < distances[0]:
                        match_x[i][j] = candidate_x
                        match_y[i][j] = candidate_y   
                    
                    wa_x = wa_x // 2
                    wa_y = wa_y // 2
        
        iteration += 1
    
    return match_x, match_y

In [30]:
a = np.array([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]])

b = np.array([[ 2,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]])

In [31]:
compare_matches(a,b, [1,1], [1,1], 3)

4